# SALBP-1

In a simple assembly line balancing problem to minimize the number of stations (SALBP-1), we are given the set of tasks $N = \{ 0, ..., n - 1 \}$.
We want to schedule tasks to a totally ordered set of stations.
Predecessors $P_i \subseteq N$ of task $i$ must be scheduled in the same station as $i$ or earlier.
Each task $i \in N$ has processing time $t_i$, and the sum of processing times of tasks scheduled in the same station must be less than or equal to the cycle time $c$.
The objective is to minimize the number of stations to schedule all tasks.

## DP Formulation

Consider using stations one by one and scheduling tasks one by one.
Let $U$ be the set of unscheduled tasks, and $r$ be the idle time in the current station.
To schedule task $i \in U$ in the current station, $P_i \cap U = \emptyset$ and $t_i \leq r$ must hold.
After scheduling $i$, $U$ becomes $U \setminus \{ i \}$ and $r$ becomes $r - t_i$.
Therefore, we have the following DP model:

$$
\begin{align}
    \text{compute } & V(N, c) \\
    & V(U, r) = \begin{cases}
         \min\limits_{i \in U : P_i \cap U = \emptyset \land t_i \leq r} V(U \setminus \{ i \}, r - t_i) & \text{if } \exists i \in U, P_i \cap U = \emptyset \land t_i \leq r \\
         1 + V(U, c) & \text{if } \forall i \in U, P_i \cap U \neq \emptyset \lor t_i > r \\
         0 & \text{if } U = \emptyset.
    \end{cases}
\end{align}
$$

If two states $(U, r)$ and $(U, r')$ have the same set of unscheduled tasks and $r \geq r'$, $(U, r)$ leads to a better solution. Therefore,

$$
    V(U, r) \leq V(U, r') \text{ if } r \geq r'.
$$

If we ignore the predecessors and the fact that a task cannot be divided for multiple stations, we get the following lower bound:

$$
    V(U, r) \geq \left\lceil \frac{\sum_{i \in U} t_i - r}{c} \right\rceil.
$$

Consider only tasks $i$ with $t_i \geq \frac{c}{2}$ and ignore predecessors.
Each station contains at most one of tasks $j$ with $t_i > \frac{c}{2}$.
Similarly, each station contains at most two tasks with $t_i = \frac{c}{2}$, and such tasks are not scheduled in the same station having tasks with $t_i > \frac{c}{2}$.
If $r \geq \frac{c}{2}$, we can possibly use the current station to schedule the tasks.
Therefore,

$$
    V(U, r) \geq \begin{cases}
        \sum\limits_{ i \in U : t_i > \frac{c}{2} } 1 + \left\lceil \sum\limits_{i \in U : t_i = \frac{c}{2}} \frac{1}{2} \right\rceil & \text{if } r < \frac{c}{2} \\
        \sum\limits_{ i \in U : t_i > \frac{c}{2} } 1 + \left\lceil \sum\limits_{i \in U : t_i = \frac{c}{2}} \frac{1}{2} \right\rceil - 1 & \text{if } r \geq \frac{c}{2}.
    \end{cases}
$$

Similaly, if we consider only tasks $i$ with $t_i \geq \frac{c}{3}$, we get the following bound:

$$
    V(U, r) \geq \begin{cases}
        \left\lceil \sum\limits_{ i \in U : t_i > \frac{2c}{3} } 1 + \sum\limits_{i \in U : t_i = \frac{2c}{3}} \frac{2}{3} + \sum\limits_{i \in U : t_i \in (\frac{c}{3}. \frac{2c}{3})} \frac{1}{2} + \sum\limits_{i \in U : t_i = \frac{c}{3}} \frac{1}{3} \right\rceil & \text{if } r < \frac{c}{3} \\
        \left\lceil \sum\limits_{ i \in U : t_i > \frac{2c}{3} } 1 + \sum\limits_{i \in U : t_i = \frac{2c}{3}} \frac{2}{3} + \sum\limits_{i \in U : t_i \in (\frac{c}{3}. \frac{2c}{3})} \frac{1}{2} + \sum\limits_{i \in U : t_i = \frac{c}{3}} \frac{1}{3} \right\rceil - 1 & \text{if } r \geq \frac{c}{3} \\
    \end{cases}
$$

## Data

In [1]:
# Number of tasks
n = 5
# Cycle time
c = 5
# Processing times
t = [2, 2, 1, 3, 2]
# Predecessors
p = [[], [], [], [0], [1]]

## Preprocessing

In [2]:
# The weight in the first term of the second bound.
weight_2_1 = [1 if t[i] > c / 2 else 0 for i in range(n)]
# The weight in the second term of the second bound.
weight_2_2 = [1 / 2 if t[i] == c / 2 else 0 for i in range(n)]
# The weight in the third bound (truncated to three decimal points).
weight_3 = [
    1.0
    if t[i] > c * 2 / 3
    else 2 / 3 // 0.001 * 1000
    if t[i] == c / 3
    else 0.5
    if t[i] > c / 3
    else 1 / 3 // 0.001 * 1000
    if t[i] == c / 3
    else 0.0
    for i in range(n)
]

## Modeling

In [3]:
# !pip install didppy

In [4]:
import math

import didppy as dp


model = dp.Model()

task = model.add_object_type(number=n)

# U
unscheduled = model.add_set_var(object_type=task, target=list(range(n)))
# r
idle_time = model.add_int_resource_var(target=0, less_is_better=False)

processing_time = model.add_int_table(t)
predecessors = model.add_set_table(p, object_type=task)

for i in range(n):
    schedule = dp.Transition(
        name="schedule {}".format(i),
        cost=dp.IntExpr.state_cost(),
        effects=[
            (unscheduled, unscheduled.remove(i)),
            (idle_time, idle_time - processing_time[i]),
        ],
        preconditions=[
            unscheduled.contains(i),
            unscheduled.isdisjoint(predecessors[i]),
            processing_time[i] <= idle_time,
        ],
    )
    model.add_transition(schedule)

open_new = dp.Transition(
    name="open a new station",
    cost=1 + dp.IntExpr.state_cost(),
    effects=[(idle_time, c)],
    preconditions=[
        ~unscheduled.contains(i)
        | ~unscheduled.isdisjoint(predecessors[i])
        | (processing_time[i] > idle_time)
        for i in range(n)
    ],
)
model.add_transition(open_new, forced=True)

model.add_base_case([unscheduled.is_empty()])

model.add_dual_bound(
    math.ceil((processing_time[unscheduled] - idle_time) / c)
)

weight_2_1_table = model.add_int_table(weight_2_1)
weight_2_2_table = model.add_float_table(weight_2_2)
model.add_dual_bound(
    weight_2_1_table[unscheduled]
    + math.ceil(weight_2_2_table[unscheduled])
    - (idle_time >= c / 2).if_then_else(1, 0)
)

weight_3_table = model.add_float_table(weight_3)
model.add_dual_bound(
    math.ceil(weight_3_table[unscheduled])
    - (idle_time >= c / 3).if_then_else(1, 0)
)

## Solving

In [5]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

open a new station
schedule 0
schedule 1
schedule 2
open a new station
schedule 3
schedule 4

Cost: 2
